# Análisis por Nacionalidades



In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Creo un diccionario para almacenar los DataFrames de cada hoja
dfs = {}

# Leo el archivo y obtengo los nombres de las hojas
with pd.ExcelFile("sexo_pais_random.xlsx") as xlsx:
    nombres_hojas = xlsx.sheet_names

# Itero sobre las hojas y las cargo en el diccionario
for hoja in nombres_hojas:
    dfs[hoja] = pd.read_excel('sexo_pais_random.xlsx', sheet_name=hoja)


In [3]:
# Guardo el nombre de las columnas para obtener las nacionalidades
nacionalidades = dfs['2021'].columns.to_list()

In [4]:
## Funcion para obtener el porcentaje de poblacion
def porcentajes(ano):
  # Selecciono df segun el año recibido
  df = dfs[ano]

  # Calculo la cantidad total de personas por país
  total_personas = df[nacionalidades].sum()

  # Calculo el porcentaje de personas por país
  porcentajes = round((total_personas / total_personas.sum()) * 100, 1)

  # Creo el DataFrame para el gráfico
  datos = pd.DataFrame({
      "Paises": nacionalidades,
      "Porcentaje": porcentajes
  })

  # Retorno el DataFrame sin sus indices
  return datos.reset_index(drop=True)


In [5]:
porcentajes('2023')

,Paises,Porcentaje
0,Chilena,12.7
1,Venezolana,12.8
2,Boliviana,12.5
3,Colombiana,12.8
4,Peruana,12.8
5,Haitiana,11.9
6,Ecuatoriana,11.6
7,Argentina,12.8


In [6]:
## Funcion para obtener las nacionalidades chilenas vs extranjeros
def cl_vs_ext(ano):
  # Selecciono df segun el año
  df = dfs[ano]

  # Calculo el total de personas chilenas
  total_chilenos = df["Chilena"].sum()

  # Calculo el total de personas extranjeras
  total_extranjeros = df[nacionalidades[1:]].sum().sum()

  # Creo un nuevo DataFrame con los totales
  df_final = pd.DataFrame({
      "Categoria": ["Chilena","Extranjeros"],
      "Valores": [total_chilenos,total_extranjeros]
      })

  return df_final


In [7]:
## Funcion para obtener distribución por sexo
def dist_sexo(ano):
  # Selecciono df segun el año
  df = dfs[ano]

  # Cuento el total de hombres
  total_hombres = np.count_nonzero(df == 0)
  # Cuento el total de mujeres
  total_mujeres = np.count_nonzero(df == 1)

  # Creo un DataFrame para retornar
  df_final = pd.DataFrame({
        "Categoria": ["Hombre","Mujer"],
        "Valores": [total_hombres,total_mujeres]
        })
  return df_final

In [8]:
# Importo librerias para el dashboard
import panel as pn
import panel.widgets as pnw
import plotly.express as px
pn.extension('plotly')

In [9]:
# Creo un selector con los años
anio = pn.widgets.Select( options= nombres_hojas,
                        value='2023',
                        margin=(18, 0, 0, 0))

In [10]:
# Diagrama de barras con la nacionalidad

# Guardo colores para las barras del grafico
colors = ['#6DE8C9', '#6DB7E8', '#6DDEE8', '#6DE8A0', '#6D90E8', '#8690FF', '#30BFDD','#C5A7FC']

@pn.depends(anio=anio)
def barra(anio):
    load_display('on')
    datos = porcentajes(anio) # Asigno los datos segun el año seleccionado

    # Configuro grafico de barras
    fig = px.bar(
        datos.sort_values(by='Porcentaje', ascending=True),
        x="Porcentaje",
        y="Paises",
        title="Diagrama de barras con la nacionalidad",
        text=datos["Porcentaje"].astype(str) + "%",
        template="plotly_dark",
        color= datos["Paises"],
        color_discrete_sequence=colors,
        orientation='h',
        range_x=[0, 15])
    fig.update_layout(title_x=0.5,xaxis={"showticklabels": False}, yaxis_title=None, showlegend=False)
    fig.update_traces(textposition='outside')
    load_display('off')
    return fig

In [11]:
# Pie nacionalidad chilena vs extranjeros
@pn.depends(anio=anio)
def pie(anio):
    datos = cl_vs_ext(anio)  # Asigno los datos segun el año seleccionado

    # Configuro grafico de pie
    fig = px.pie(datos,values="Valores", names="Categoria",  title="Nacionalidad",template="plotly_dark", color = datos['Categoria'],
        color_discrete_map = {'Chilena': '#30BFDD', 'Extranjeros': '#8690FF'})
    fig.update_traces(textposition='outside')
    fig.update_layout(title_x=0.5,legend_orientation='h')
    return fig

In [12]:
# Anillo Distribucion por sexo
@pn.depends(anio=anio)
def anillo(anio):
    datos = dist_sexo(anio) # Asigno los datos segun el año seleccionado

    # Configuro grafico de pie
    fig = px.pie(
        datos,values="Valores",
        names="Categoria",
        hole=0.6,
        title="Sexo",
        template="plotly_dark",
        color = datos['Categoria'],
        color_discrete_map = {'Hombre': '#30BFDD', 'Mujer': '#8690FF'}
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(title_x=0.5, legend_orientation='h', title_xanchor='right')
    return fig

In [13]:
#Agrego widget para indicar la carga de los datos
loading = pn.indicators.LoadingSpinner(value=False, size=30, visible=False)

#Funcion para activar o desactivar el widget
def load_display(x):
    if(x=='on'):
        loading.value=True
        loading.visible=True
    if(x=='off'):
        loading.value=False
        loading.visible=False

In [22]:
# Configuro diseño del panel
pn.config.sizing_mode='stretch_width'
pn.config.theme='dark'

# Asigno Titulo y graficos en el panel
pn.Column(
   '# Análisis por Nacionalidad',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    pn.Row('# Año:', anio, width=390),
    pn.Row(loading),
    pn.Row(barra,pie,anillo)
)

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Divider(margin=(-20, 0, 0, 0), sizing_mode='stretch_width')
    [2] Row(width=390)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Select(margin=(18, 0, 0, 0), options=['2021', '2022', '2023'], value='2022')
    [3] Row(sizing_mode='stretch_width')
        [0] LoadingSpinner(size=30, visible=False)
    [4] Row(sizing_mode='stretch_width')
        [0] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')
        [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')